# Initial Development for CIFAR-10 Image Classification using GUNN-15



In [0]:
#!pip install tensorflow==1.14.0
!pip install -U keras

     |████████████████████████████████| 378kB 2.8MB/s 
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5


In [0]:
import numpy as np
import keras
from keras import layers
from keras.datasets import cifar10
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.layers.core import Lambda
from keras import regularizers
from keras.models import Model, Sequential
from keras import optimizers
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
#from kt_utils import *
#from keras.backend import tf
from keras.layers import Lambda
import tensorflow as tf
 



import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline

Using TensorFlow backend.


In [0]:
(X_train_orig, Y_train_orig), (X_test_orig, Y_test_orig) = cifar10.load_data()
X_train_orig = X_train_orig.astype('float32')
X_test_orig = X_test_orig.astype('float32')

# Normalize image vectors
mean = np.mean(X_train_orig,axis=(0,1,2,3))
std = np.std(X_train_orig, axis=(0, 1, 2, 3))
X_train = (X_train_orig-mean)/(std+1e-7)
X_test = (X_test_orig-mean)/(std+1e-7)
#X_train = X_train_orig/255.
#X_test = X_test_orig/255.

Y_train = keras.utils.to_categorical(Y_train_orig, 10)
Y_test = keras.utils.to_categorical(Y_test_orig, 10)

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

170500096/170498071 [==============================] - 2s 0us/step
number of training examples = 50000
number of test examples = 10000
X_train shape: (50000, 32, 32, 3)
Y_train shape: (50000, 10)
X_test shape: (10000, 32, 32, 3)
Y_test shape: (10000, 10)


In [0]:
def Gunn2D_Backup(X, input_channels, expansion_rate):
    """
    Implementation of the Gunn2D layer as defined in the paper
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    input_channels -- integer, defining the number of input filters 
    expansion_rate -- integer, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the Gunn2D layer, tensor of shape (n_H, n_W, n_C)
    """
    output_channels = input_channels / expansion_rate
    
    for i in range(1, expansion_rate+1):
      X_shortcut = X
      print(X)
      X = Conv2D(output_channels*2, (1, 1), strides = (1, 1), padding='valid', name = 'g1'+str(i))(X)
      print(X)
      X = BatchNormalization(axis = 3 , name = 'bn5')(X)
      X = Activation('relu')(X)
      X = Conv2D(output_channels*2, (3, 3), strides = (1, 1), padding='same', name = 'g2'+str(i))(X)
      X = BatchNormalization(axis = 3 , name = 'bn6')(X)
      X = Activation('relu')(X)
      X = Conv2D(output_channels, (1, 1), strides = (1, 1), padding='valid', name = 'g3'+str(i))(X)
      X = BatchNormalization(axis = 3 , name = 'bn7')(X)
      print(X)
      #Implementation of the identity block in Residual Network
      X_shortcut = Conv2D(output_channels, (1, 1), strides = (1, 1), padding='valid', name = 'g1'+str(i))(X)
      X_shortcut = BatchNormalization(axis = 3 , name = 'bn5')(X)
      X = Add()([X , X_shortcut])

    return X
  

In [0]:
def Gunn2D_back2(X, input_channels, expansion_rate):
    """
    Implementation of the Gunn2D layer as defined in the paper
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    input_channels -- integer, defining the number of input filters 
    expansion_rate -- integer, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the Gunn2D layer, tensor of shape (n_H, n_W, n_C)
    """
    output_channels = input_channels / expansion_rate
    
    for i in range(1, expansion_rate+1):
      X_shortcut = X
      print(X)
      X = Conv2D(output_channels*2, (1, 1), strides = (1, 1), padding='valid', name = 'g1'+str(i))(X)
      print(X)
      X = BatchNormalization(axis = 3 , name = 'bn5')(X)
      X = Activation('relu')(X)
      X = Conv2D(output_channels*2, (3, 3), strides = (1, 1), padding='same', name = 'g2'+str(i))(X)
      X = BatchNormalization(axis = 3 , name = 'bn6')(X)
      X = Activation('relu')(X)
      X = Conv2D(output_channels, (1, 1), strides = (1, 1), padding='valid', name = 'g3'+str(i))(X)
      X = BatchNormalization(axis = 3 , name = 'bn7')(X)
      print(X)
      #Implementation of the identity block in Residual Network
      X_shortcut = Conv2D(output_channels, (1, 1), strides = (1, 1), padding='valid', name = 'g1'+str(i))(X)
      X_shortcut = BatchNormalization(axis = 3 , name = 'bn5')(X)
      X = Add()([X , X_shortcut])

    return X
  
def Gunn2D(X, input_channels, expansion_rate):  # lambda layer testing
    X_shortcut = X
    X = Add()([X , X_shortcut])
    return X


#  Convolutional Neural Networks - Forward and Backward pass






In [0]:
def zero_pad(X, pad):
    """
    Pad with zeros all images of the dataset X. The padding is applied to the height and width of an image, 
    as illustrated in Figure 1.
    
    Argument:
    X -- python numpy array of shape (m, n_H, n_W, n_C) representing a batch of m images
    pad -- integer, amount of padding around each image on vertical and horizontal dimensions
    
    Returns:
    X_pad -- padded image of shape (m, n_H + 2*pad, n_W + 2*pad, n_C)
    """
    
    X_pad = np.pad(X, ((0,0), (pad,pad), (pad,pad), (0,0)), 'constant', constant_values = (0,0))
    
    return X_pad

def conv_single_step(a_slice_prev, W, b):
    """
    Apply one filter defined by parameters W on a single slice (a_slice_prev) of the output activation 
    of the previous layer.
    
    Arguments:
    a_slice_prev -- slice of input data of shape (f, f, n_C_prev)
    W -- Weight parameters contained in a window - matrix of shape (f, f, n_C_prev)
    b -- Bias parameters contained in a window - matrix of shape (1, 1, 1)
    
    Returns:
    Z -- a scalar value, result of convolving the sliding window (W, b) on a slice x of the input data
    """

    # Element-wise product between a_slice and W. Do not add the bias yet.
    s = a_slice_prev * W   # element wise product in python
    # Sum over all entries of the volume s.
    Z = np.sum(s)
    # Add bias b to Z. Cast b to a float() so that Z results in a scalar value.
    Z =  Z + float(b)

    return Z


def conv_forward(A_prev, W, b, hparameters):
    """
    Implements the forward propagation for a convolution function
    
    Arguments:
    A_prev -- output activations of the previous layer, numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
    W -- Weights, numpy array of shape (f, f, n_C_prev, n_C)
    b -- Biases, numpy array of shape (1, 1, 1, n_C)
    hparameters -- python dictionary containing "stride" and "pad"
        
    Returns:
    Z -- conv output, numpy array of shape (m, n_H, n_W, n_C)
    cache -- cache of values needed for the conv_backward() function
    """
    
    # Retrieve dimensions from A_prev's shape 
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    print('Forward prop : {}'.format(A_prev.shape))
    
    # Retrieve dimensions from W's shape 
    (f, f, n_C_prev, n_C) = W.shape
    
    # Retrieve information from "hparameters" 
    stride = hparameters["stride"]
    pad = hparameters["pad"]
    
    # Compute the dimensions of the CONV output volume using the formula given above.
    n_H = int((n_H_prev + 2*pad -f) // stride ) + 1
    n_W = int((n_W_prev + 2*pad -f) // stride ) + 1
    
    print('Input Shape: {} {} {} {} '.format(m, n_H_prev, n_W_prev, n_C_prev, ))
    print('Filter Shape: {} {} {} {} '.format(f, f, n_C_prev, n_C))
    print('output Shape: {} {} {} {} '.format(n_H, n_W, stride, pad))
    # Initialize the output volume Z with zeros. 
    #Z = np.zeros(( m, n_H, n_W, n_C ))
    #Z = tf.zeros(( m, n_H, n_W, n_C ), tf.float32)
    Z = tf.Variable(tf.zeros(( m, n_H, n_W, n_C ), tf.float32), validate_shape=False)
    
    # Create A_prev_pad by padding A_prev
    A_prev_pad = zero_pad(A_prev,pad)

    for i in range(m):                               # loop over the batch of training examples
        a_prev_pad = A_prev_pad[i]                               # Select ith training example's padded activation
        for h in range(n_H):                           # loop over vertical axis of the output volume    
            for w in range(n_W):                       # loop over horizontal axis of the output volume
                for c in range(n_C):                   # loop over channels (= #filters) of the output volume
                    
                    # Find the corners of the current "slice" 
                    vert_start = h*stride
                    vert_end = vert_start+f
                    horiz_start = w*stride
                    horiz_end = horiz_start+f
                    
                    # Use the corners to define the (3D) slice of a_prev_pad 
                    a_slice_prev = a_prev_pad[ vert_start:vert_end, horiz_start:horiz_end, : ]
                    
                    # Convolve the (3D) slice with the correct filter W and bias b, to get back one output neuron. 
                    Z[i, h, w, c] = conv_single_step(a_slice_prev, W[:,:,:,c], b[:,:,:,c])

                                            
    # Making sure your output shape is correct
    assert(Z.shape == (m, n_H, n_W, n_C))
    
    # Save information in "cache" for the backprop
    cache = (A_prev, W, b, hparameters)
    
    return Z, cache




def conv_backward(dZ, cache):
    """
    Implement the backward propagation for a convolution function
    
    Arguments:
    dZ -- gradient of the cost with respect to the output of the conv layer (Z), numpy array of shape (m, n_H, n_W, n_C)
    cache -- cache of values needed for the conv_backward(), output of conv_forward()
    
    Returns:
    dA_prev -- gradient of the cost with respect to the input of the conv layer (A_prev),
               numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
    dW -- gradient of the cost with respect to the weights of the conv layer (W)
          numpy array of shape (f, f, n_C_prev, n_C)
    db -- gradient of the cost with respect to the biases of the conv layer (b)
          numpy array of shape (1, 1, 1, n_C)
    """
    
    # Retrieve information from "cache"
    (A_prev, W, b, hparameters) = cache
    
    # Retrieve dimensions from A_prev's shape
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    
    # Retrieve dimensions from W's shape
    (f, f, n_C_prev, n_C) = W.shape
    
    # Retrieve information from "hparameters"
    stride = hparameters['stride']
    pad = hparameters['pad']
    
    # Retrieve dimensions from dZ's shape
    (m, n_H, n_W, n_C) = dZ.shape
    
    # Initialize dA_prev, dW, db with the correct shapes
    dA_prev = np.zeros((m, n_H_prev, n_W_prev, n_C_prev))                           
    dW = np.zeros((f, f, n_C_prev, n_C))
    db = np.zeros((1, 1, 1, n_C))

    # Pad A_prev and dA_prev
    A_prev_pad = zero_pad(A_prev, pad)
    dA_prev_pad = zero_pad(dA_prev, pad)
    
    for i in range(m):                       # loop over the training examples
        
        # select ith training example from A_prev_pad and dA_prev_pad
        a_prev_pad = A_prev_pad[i, :]
        da_prev_pad = dA_prev_pad[i, :]
        
        for h in range(n_H):                   # loop over vertical axis of the output volume
            for w in range(n_W):               # loop over horizontal axis of the output volume
                for c in range(n_C):           # loop over the channels of the output volume
                    
                    # Find the corners of the current "slice"
                    vert_start = h*stride
                    vert_end = vert_start+f
                    horiz_start = w*stride
                    horiz_end = horiz_start+f
                    
                    # Use the corners to define the slice from a_prev_pad
                    a_slice = a_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :]

                    # Update gradients for the window and the filter's parameters 
                    da_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :] += W[:,:,:,c] * dZ[i, h, w, c]
                    dW[:,:,:,c] += a_slice * dZ[i, h, w, c]
                    db[:,:,:,c] += dZ[i, h, w, c]
                    
        # Set the ith training example's dA_prev to the unpaded da_prev_pad : use X[pad:-pad, pad:-pad, :]
        dA_prev[i, :, :, :] = dA_prev_pad[i, pad:-pad, pad:-pad, :]
    
    # Making sure your output shape is correct
    assert(dA_prev.shape == (m, n_H_prev, n_W_prev, n_C_prev))
    
    return dA_prev, dW, db

In [0]:
#A_prev = np.random.randn(10,4,4,3)
#W = np.random.randn(2,2,3,8)
#b = np.random.randn(1,1,1,8)
#hparameters = {"pad" : 2,
#               "stride": 2}

#Z, cache_conv = conv_forward(A_prev, W, b, hparameters)
#dA, dW, db = conv_backward(Z, cache_conv)



# Load pretrained VGG-16 for ImageNet in order to perform Transfer learning

> Instead of random initializing we can initialize our model using weights of model learned for ImageNet and then further train our GUNN-15 model using random initialization.



## Building GUNN-15 Model in Keras for 10 classes

In [0]:
def GUNN_15_model(input_shape):
    """
    Implementation of the GUNN-15 Model.
    
    Arguments:
    input_shape -- shape of the images of the dataset

    Returns:
    model -- a Model() instance in Keras
    """
    print("input_shape: {}".format(input_shape))
    X_input = Input(input_shape)
    print("X_input: {}".format(X_input.shape))
    #frozen = VGG16 (weights="imagenet", input_shape=(32,32,3), include_top=False)
    #trainable = frozen.output
    X = Conv2D(64, (3, 3), strides = (1, 1), padding='same', name = 'z1')(X_input) # 32x32x3 -> 32x32x64   ; padding = 1
    X = BatchNormalization(axis = 3 , name = 'bn1')(X)
    X = Activation('relu')(X)
    convlayer = Conv2D(240, (1, 1), strides = (1, 1), padding='valid', name = 'z2')
    X = convlayer(X) # 32x32x64 -> 32x32x240
    #print(convlayer.get_weights())
    layer = BatchNormalization(axis = 3 , name = 'bn2')
    X = layer(X)
    X = Activation('relu')(X)
    
    #print(layer.get_weights())
    #X = layer(X)
    #print(layer.get_weights())
    print(X)
    weights = layer.get_weights()
    gunnlayer = Gunn2D(240, 20) # custom Keras layer class
    #print(gunnlayer.get_weights())
    X = gunnlayer(X)
    #print(gunnlayer.get_weights())
    #X = Gunn2D(X, 240, 20, weights[:-1], weights[-1])
    #X = Gunn2D(X, 240, 20, tf.convert_to_tensor(weights[:-1], dtype=tf.float32), tf.convert_to_tensor(weights[-1], dtype=tf.float32))
    #X = Lambda(lambda X: Gunn2D(X, 240, 20, tf.convert_to_tensor(weights[:-1], dtype=tf.float32), tf.convert_to_tensor(weights[-1], dtype=tf.float32)))(X)
    #X = Lambda(Gunn2D(X, 240, 20))(X)
    print(X)
    X = Conv2D(300, (1, 1), strides = (1, 1), padding='valid', name = 'z3')(X)
    X = BatchNormalization(axis = 3 , name = 'bn3')(X)
    X = Activation('relu')(X)
    X = AveragePooling2D((2, 2), name = 'avg_pool1')(X)
    print(X)
    X = Lambda(lambda x: Gunn2D(X, 300, 25))(X)
    print(X)
    X = Conv2D(360, (1, 1), strides = (1, 1), padding='valid', name = 'z4')(X)
    X = BatchNormalization(axis = 3 , name = 'bn3')(X)
    X = Activation('relu')(X)
    X = AveragePooling2D((2, 2), name = 'avg_pool2')(X)
    print(X)
    X = Lambda(lambda x: Gunn2D(X, 360, 30))(X)
    print(X)
    X = Conv2D(360, (1, 1), strides = (1, 1), padding='valid', name = 'z5')(X)
    X = BatchNormalization(axis = 3 , name = 'bn3')(X)
    X = Activation('relu')(X)
    X = AveragePooling2D((8, 8), name = 'avg_pool3')(X)
    X = Flatten()(X)
    X = Dense(360, activation='softmax', name = 'fc1')(X)
    X = Dense(360, activation='softmax', name = 'fc2')(X)
    X = Dense(10, activation='softmax', name = 'fc3')(X)

    model = Model(inputs = X_input, outputs = X, name = 'GUNN-15-Model')
    print(model)
    return model

## GUNN Layer implementation

In [0]:
class Gunn2D(tf.keras.layers.Layer):
  def __init__(self, input_channels, expansion_rate):
    super(Gunn2D, self).__init__()
    self.input_channels = input_channels
    self.expansion_rate = expansion_rate
    self.hparameters = {"pad" : 0, "stride": 1}

  def build(self, input_shape):
      self.w = self.add_weight(shape=(1, 1, self.input_channels),
                             initializer='random_normal',
                             trainable=True)
      self.b = self.add_weight(shape=(self.input_channels,),
                             initializer='zeros',
                             trainable=True)
    
  def call(self, inputs):
    print('inputs')
    print(inputs)
    #(m, n_H_prev, n_W_prev, n_C_prev) = inputs.shape
    inputs = Conv2D(240, (1, 1), strides = (1, 1), padding='valid', name = 'zGunn')(inputs)
    #conv_forward(inputs, self.w, self.b, self.hparameters) # error: values not enough:  (f, f, n_C_prev, n_C) = W.shape
    return inputs 


@tf.custom_gradient
def Gunn2D_3(A_prev, input_channels, expansion_rate, W, b):
    def grad(dZ):
        print('backpropagation')
        dA, dW, db  = conv_backward(dZ, cache)
        return dA

    print('custom_gradient : {},  weights: '.format(A_prev.shape, weights))
    Z, cache = conv_forward(A_prev, W, b, hparameters)
    Z = tf.cast(Z, 'float32')
    return Z, grad

In [0]:
# Create model
#gunn15model = GUNN_15_model(X_train.shape) # input: (32, 32, 3)
#print(X_train.shape) # (50000, 32, 32, 3)
gunn15model = GUNN_15_model(X_train.shape[1:]) # input: (32, 32, 3)




input_shape: (32, 32, 3)
X_input: (None, 32, 32, 3)
Tensor("activation_16/Relu:0", shape=(None, 32, 32, 240), dtype=float32)
inputs
Tensor("activation_16/Relu:0", shape=(None, 32, 32, 240), dtype=float32)


AttributeError: ignored

In [0]:
print(X_train.shape)
print(type(X_train.shape[1]))

(50000, 32, 32, 3)
<class 'int'>


In [0]:
print(y_train)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]
